In [35]:
from pymongo import MongoClient
import time
from pymongo import ASCENDING, DESCENDING

In [36]:
client = MongoClient("mongodb://localhost:27018/")
db = client["anime_database"]

## JOVO GUŽVIĆ, IN61/2020

### Za svaki žanr, pronaći anime sa najvećom prosečnom ocenom u tom žanru

In [67]:
pipeline1_opt = [
    {"$unwind": "$genre"},
    {"$group": {
        "_id": {
            "genre": "$genre",
            "anime_uid": "$uid",
            "title": "$title",
            "average_score": "$average_score"  
        },
        "max_average_score": {"$max": "$average_score"}
    }},
    {"$sort": {
        "_id.genre": 1,
        "max_average_score": -1
    }},
    {"$group": {
        "_id": "$_id.genre",
        "top_anime": {"$first": {
            "anime_uid": "$_id.anime_uid",
            "title": "$_id.title",
            "max_average_score": "$max_average_score"
        }}
    }},
    {"$sort": {
        "_id": 1
    }},
    {"$project": {
        "_id": 0,
        "genre": "$_id",
        "anime_uid": "$top_anime.anime_uid",
        "title": "$top_anime.title",
        "max_average_score": "$top_anime.max_average_score"
    }}
]


# Measure query execution time
start_time = time.time()
result1_opt = list(db.animes_opt.aggregate(pipeline1_opt))
end_time = time.time()
duration = end_time - start_time


# Print results and query duration
print("Top anime with highest average review score for each genre (optimized):", result1_opt)
print("Query duration:", duration, "seconds")

Top anime with highest average review score for each genre (optimized): [{'genre': 'Action', 'anime_uid': 38084, 'title': 'Fate/Grand Order: Zettai Majuu Sensen Babylonia', 'max_average_score': 10.0}, {'genre': 'Adventure', 'anime_uid': 996, 'title': 'Bishoujo Senshi Sailor Moon: Sailor Stars', 'max_average_score': 9.8}, {'genre': 'Cars', 'anime_uid': 388, 'title': 'Capeta', 'max_average_score': 9.0}, {'genre': 'Comedy', 'anime_uid': 22673, 'title': 'Kuroko no Basket 2nd Season NG-shuu', 'max_average_score': 10.0}, {'genre': 'Dementia', 'anime_uid': 934, 'title': 'Higurashi no Naku Koro ni', 'max_average_score': 8.625}, {'genre': 'Demons', 'anime_uid': 1253, 'title': 'Saint Seiya: Meiou Hades Meikai-hen', 'max_average_score': 9.0}, {'genre': 'Drama', 'anime_uid': 38250, 'title': 'Xiao Lu He Xiao Lan', 'max_average_score': 10.0}, {'genre': 'Ecchi', 'anime_uid': 268, 'title': 'Golden Boy', 'max_average_score': 8.542857142857143}, {'genre': 'Fantasy', 'anime_uid': 38084, 'title': 'Fate/Gr

### Za svaku godinu izračunaj prosek svih prosečnih ocena animea koji su počeli emitovanje date godine

In [69]:
# Define the aggregation pipeline for optimized collections
pipeline2_opt = [
    {"$match": {
        "start_years": {"$gt": 2000}
    }},
    {"$group": {
        "_id": "$start_years",
        "average_overall": {"$avg": "$average_score"},
        "average_story": {"$avg": "$average_story"},
        "average_animation": {"$avg": "$average_animation"},
        "average_sound": {"$avg": "$average_sound"},
        "average_character": {"$avg": "$average_character"},
        "average_enjoyment": {"$avg": "$average_enjoyment"}
    }},
    {"$project": {
        "_id": 1,
        "average_all": {
            "$avg": [
                "$average_overall",
                "$average_story",
                "$average_animation",
                "$average_sound",
                "$average_character",
                "$average_enjoyment"
            ]
        }
    }},
    {"$sort": {
        "_id": 1
    }}
]


# Measure query execution time
start_time = time.time()
result2_opt = list(db.animes_opt.aggregate(pipeline2_opt))
end_time = time.time()
duration = end_time - start_time

# Print results and query duration
print("Average of all average metrics for each year (optimized):", result2_opt)
print("Query duration:", duration, "seconds")


Average of all average metrics for each year (optimized): [{'_id': 2001.0, 'average_all': 8.203492063492064}, {'_id': 2002.0, 'average_all': 8.061814726452885}, {'_id': 2003.0, 'average_all': 8.390571782963088}, {'_id': 2004.0, 'average_all': 6.805144105144105}, {'_id': 2005.0, 'average_all': 8.318298059964727}, {'_id': 2006.0, 'average_all': 8.033311352128557}, {'_id': 2007.0, 'average_all': 8.296320346320346}, {'_id': 2008.0, 'average_all': 7.8821223270040415}, {'_id': 2009.0, 'average_all': 7.30168473808245}, {'_id': 2010.0, 'average_all': 8.052931912142439}, {'_id': 2011.0, 'average_all': 8.316095112970112}, {'_id': 2012.0, 'average_all': 7.687615828832104}, {'_id': 2013.0, 'average_all': 7.86227207136298}, {'_id': 2014.0, 'average_all': 7.640889539654245}, {'_id': 2015.0, 'average_all': 7.558856942091055}, {'_id': 2016.0, 'average_all': 7.151043169793169}, {'_id': 2017.0, 'average_all': 7.9352931758343415}, {'_id': 2018.0, 'average_all': 7.944210102960103}, {'_id': 2019.0, 'averag

### Pronađi top 3 animea koji imaju najveći prosek zbira Enjoyment i Sound ocena

In [39]:
pipeline3 = [
    {
        "$lookup": {
            "from": "animes_opt",
            "localField": "anime_uid",
            "foreignField": "uid",
            "as": "anime_info"
        }
    },
    {
        "$unwind": "$anime_info"
    },
    {
        "$unwind": "$anime_info.genre"
    },
    {
        "$group": {
            "_id": "$anime_info.genre",
            "average_enjoyment_sound": {"$avg": {"$avg": ["$scores.Enjoyment", "$scores.Sound"]}}
        }
    },
    {
        "$sort": {
            "average_enjoyment_sound": -1
        }
    },
    {
        "$limit": 3
    }
]

start_time = time.time()

result3 = db.reviews_opt.aggregate(pipeline3)

end_time = time.time()
duration = end_time - start_time

for record in result3:
    print(f"Genre: {record['_id']}, Average Enjoyment+Sound Score: {record['average_enjoyment_sound']}")
print("Query duration:", duration, "seconds")

Genre: Cars, Average Enjoyment+Sound Score: 8.6875
Genre: Josei, Average Enjoyment+Sound Score: 8.382978723404255
Genre: Parody, Average Enjoyment+Sound Score: 8.328260869565218
Query duration: 0.2763948440551758 seconds


### Za svaki žanr prikaži pol koji je animee u tom žanru ocenio najvećom prosečnom ocenom

In [41]:
pipeline4 = [
  {
    "$lookup": {
      "from": "animes_opt",
      "localField": "anime_uid",
      "foreignField": "uid",
      "as": "anime_info"
    }
  },
  {
    "$unwind": "$anime_info"
  },
  {
    "$lookup": {
      "from": "profiles_opt",
      "localField": "profile",
      "foreignField": "profile",
      "as": "profile_info"
    }
  },
  {
    "$unwind": "$profile_info"
  },
  {
    "$unwind": "$anime_info.genre"
  },
  {
    "$group": {
      "_id": {
        "genre": "$anime_info.genre",
        "gender": "$profile_info.gender"
      },
      "average_score": { "$avg": "$score" }
    }
  },
  {
    "$sort": {
      "_id.genre": 1,
      "average_score": -1
    }
  },
  {
    "$group": {
      "_id": "$_id.genre",
      "highest_avg_gender": { "$first": "$_id.gender" },
      "highest_avg_score": { "$first": "$average_score" }
    }
  },
  {
    "$sort": {
      "_id": 1
    }
  }
]

start_time = time.time()

# Execute the aggregation pipeline
result4 = db.reviews_opt.aggregate(pipeline4)

end_time = time.time()
duration = end_time - start_time

# Displaying the results
for record in result4:
    print(f"Genre: {record['_id']}, Gender: {record['highest_avg_gender']}, Average Score: {record['highest_avg_score']}")
print("Query duration:", duration, "seconds")

Genre: Action, Gender: Female, Average Score: 8.251984126984127
Genre: Adventure, Gender: Non-Binary, Average Score: 8.266666666666667
Genre: Cars, Gender: Male, Average Score: 8.5
Genre: Comedy, Gender: Female, Average Score: 8.330275229357799
Genre: Dementia, Gender: Male, Average Score: 8.19543973941368
Genre: Demons, Gender: Female, Average Score: 8.526666666666667
Genre: Drama, Gender: Female, Average Score: 8.333333333333334
Genre: Ecchi, Gender: Male, Average Score: 8.10909090909091
Genre: Fantasy, Gender: Non-Binary, Average Score: 8.533333333333333
Genre: Game, Gender: Male, Average Score: 7.4873096446700504
Genre: Harem, Gender: Male, Average Score: 9.071428571428571
Genre: Historical, Gender: Female, Average Score: 8.766233766233766
Genre: Horror, Gender: Female, Average Score: 8.271186440677965
Genre: Josei, Gender: Non-Binary, Average Score: 9.5
Genre: Kids, Gender: Male, Average Score: 8.08695652173913
Genre: Magic, Gender: Non-Binary, Average Score: 9.0
Genre: Martial Ar

### Za svako godiše, pronađi anime koji se najčešće pojavljuje među omiljenim i izračunaj njegovu prosečnu ocenu, prosek dodatnih ocena, razliku njih dve.
### Prikaži samo anime kod koji je ta razlika veća od nula i sortiraj po godini rođenja

In [42]:
pipeline5 = [
    {"$unwind": "$favorites_anime"},
    {"$group": {
        "_id": {
            "birthday": "$birthday",
            "anime_uid": "$favorites_anime"
        },
        "count": {"$sum": 1}
    }},
    {"$sort": {
        "_id.birthday": 1,
        "count": -1
    }},
    {"$group": {
        "_id": "$_id.birthday",
        "top_anime_uid": {"$first": "$_id.anime_uid"},
        "count": {"$first": "$count"}
    }},
    {"$lookup": {
        "from": "animes_opt",
        "localField": "top_anime_uid",
        "foreignField": "uid",
        "as": "anime_details"
    }},
    {"$unwind": "$anime_details"},
    {"$project": {
        "_id": 0,
        "birthday": "$_id",
        "top_anime_title": "$anime_details.title",
        "top_anime_uid": 1,
        "count": 1
    }},
    {"$sort": {
        "count": -1
    }},
    {"$lookup": {
        "from": "reviews_opt",
        "localField": "top_anime_uid",
        "foreignField": "anime_uid",
        "as": "reviews"
    }},
    {"$unwind": {
        "path": "$reviews",
        "preserveNullAndEmptyArrays": True
    }},
    {"$group": {
        "_id": {
            "birthday": "$birthday",
            "top_anime_title": "$top_anime_title",
            "top_anime_uid": "$top_anime_uid",
            "count": "$count"
        },
        "average_score": {"$avg": "$reviews.score"},
        "average_story_score": {"$avg": {"$ifNull": ["$reviews.scores.Story", 0]}},
        "average_animation_score": {"$avg": {"$ifNull": ["$reviews.scores.Animation", 0]}},
        "average_sound_score": {"$avg": {"$ifNull": ["$reviews.scores.Sound", 0]}},
        "average_character_score": {"$avg": {"$ifNull": ["$reviews.scores.Character", 0]}},
        "average_enjoyment_score": {"$avg": {"$ifNull": ["$reviews.scores.Enjoyment", 0]}}
    }},
    {"$project": {
        "_id": 0,
        "birthday": "$_id.birthday",
        "top_anime_title": "$_id.top_anime_title",
        "top_anime_uid": "$_id.top_anime_uid",
        "count": "$_id.count",
        "average_score": {"$ifNull": ["$average_score", None]},
        "average_other_scores": {
            "$avg": [
                "$average_story_score",
                "$average_animation_score",
                "$average_sound_score",
                "$average_character_score",
                "$average_enjoyment_score"
            ]
        },
        "absolute_difference": {
            "$abs": {
                "$subtract": [
                    {"$ifNull": ["$average_score", 0]},
                    {"$avg": [
                        "$average_story_score",
                        "$average_animation_score",
                        "$average_sound_score",
                        "$average_character_score",
                        "$average_enjoyment_score"
                    ]}
                ]
            }
        }
    }},
    {"$match": {
        "absolute_difference": {"$gt": 0}
    }},
    {"$sort": {
        "birthday": -1
    }}
]

start_time = time.time()

result5 = list(db.profiles_opt.aggregate(pipeline5))

end_time = time.time()
duration = end_time - start_time

print(result5)
print("Query duration:", duration, "seconds")


[{'birthday': 2021.0, 'top_anime_title': 'Clannad: After Story', 'top_anime_uid': 4181, 'count': 1, 'average_score': 9.547619047619047, 'average_other_scores': 9.361904761904762, 'absolute_difference': 0.18571428571428505}, {'birthday': 2018.0, 'top_anime_title': 'Clannad: After Story', 'top_anime_uid': 4181, 'count': 3, 'average_score': 9.547619047619047, 'average_other_scores': 9.361904761904762, 'absolute_difference': 0.18571428571428505}, {'birthday': 2013.0, 'top_anime_title': 'FLCL', 'top_anime_uid': 227, 'count': 2, 'average_score': 7.983870967741935, 'average_other_scores': 7.53225806451613, 'absolute_difference': 0.45161290322580516}, {'birthday': 2007.0, 'top_anime_title': 'Fullmetal Alchemist: Brotherhood', 'top_anime_uid': 5114, 'count': 2, 'average_score': 9.326923076923077, 'average_other_scores': 9.003846153846155, 'absolute_difference': 0.32307692307692193}, {'birthday': 2006.0, 'top_anime_title': 'Fullmetal Alchemist: Brotherhood', 'top_anime_uid': 5114, 'count': 4, 'a

## Radovan Zoraja IN58/2020

In [43]:
pipeline = [
    {
        "$lookup": {
            "from": "animes_opt",
            "localField": "anime_uid",
            "foreignField": "uid",
            "as": "anime_info"
        }
    },
    {"$unwind": "$anime_info"},
    {"$match": {"anime_info.title": "Clannad: After Story", "score": {"$eq": 9}}},
    {
        "$lookup": {
            "from": "profiles_opt",
            "localField": "profile",
            "foreignField": "profile",
            "as": "profile_info"
        }
    },
    {"$unwind": "$profile_info"},
    {
        "$match": {
            "$expr": {
                "$in": ["$anime_info.uid", "$profile_info.favorites_anime"]
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "profile": 1,
            "anime_title": "$anime_info.title",
            "score": 1,
            "favorite_anime": "$profile_info.favorites_anime"
        }
    }
]

result = list(db.reviews_opt.aggregate(pipeline))
result

[{'profile': 'CalebTheMenace',
  'score': 9,
  'anime_title': 'Clannad: After Story',
  'favorite_anime': [4181,
   11617,
   1735,
   35849,
   28999,
   29803,
   31964,
   36098,
   5081,
   8525]},
 {'profile': 'CalebTheMenace',
  'score': 9,
  'anime_title': 'Clannad: After Story',
  'favorite_anime': [4181,
   11617,
   1735,
   35849,
   28999,
   29803,
   31964,
   36098,
   5081,
   8525]},
 {'profile': 'CalebTheMenace',
  'score': 9,
  'anime_title': 'Clannad: After Story',
  'favorite_anime': [4181,
   11617,
   1735,
   35849,
   28999,
   29803,
   31964,
   36098,
   5081,
   8525]},
 {'profile': 'CalebTheMenace',
  'score': 9,
  'anime_title': 'Clannad: After Story',
  'favorite_anime': [4181,
   11617,
   1735,
   35849,
   28999,
   29803,
   31964,
   36098,
   5081,
   8525]},
 {'profile': 'CalebTheMenace',
  'score': 9,
  'anime_title': 'Clannad: After Story',
  'favorite_anime': [4181,
   11617,
   1735,
   35849,
   28999,
   29803,
   31964,
   36098,
   5081,
 

In [44]:
pipeline = [
    {
        "$lookup": {
            "from": "animes_opt",
            "localField": "anime_uid",
            "foreignField": "uid",
            "as": "anime_info"
        }
    },
    {"$unwind": "$anime_info"},
    {"$match": {"anime_info.genre": "Action"}},
    {
        "$group": {
            "_id": "$profile",
            "total_reviews": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "profile": "$_id",
            "total_reviews": 1
        }
    },
    {"$sort": {"total_reviews": -1}},
    {"$limit": 5}
]
result = list(db.reviews_opt.aggregate(pipeline))
result

[{'total_reviews': 16, 'profile': 'ggultra2764'},
 {'total_reviews': 13, 'profile': 'literaturenerd'},
 {'total_reviews': 12, 'profile': 'FullmetalCowboy'},
 {'total_reviews': 12, 'profile': 'BowlingJD'},
 {'total_reviews': 9, 'profile': 'HybridMedia'}]

In [70]:
pipeline = [
    {
        "$lookup": {
            "from": "profiles_opt",
            "localField": "profile",
            "foreignField": "profile",
            "as": "profile_info"
        }
    },
    {"$unwind": "$profile_info"},
    {"$match": {"profile_info.gender": "Non-Binary"}},
    {
        "$group": {
            "_id": "$anime_uid",
            "average_score": {"$avg": "$scores.Animation"}
        }
    },
    {
        "$lookup": {
            "from": "animes_opt",
            "localField": "_id",
            "foreignField": "uid",
            "as": "anime_info"
        }
    },
    {"$unwind": "$anime_info"},
    {
        "$project": {
            "_id": 0,
            "anime_title": "$anime_info.title",
            "average_score": 1
        }
    },
    {"$sort": {"average_score": -1}},
    {"$limit": 100}
]
result = list(db.reviews_opt.aggregate(pipeline))
result


[{'average_score': 10.0, 'anime_title': 'Overlord'},
 {'average_score': 10.0, 'anime_title': 'Colorful (Movie)'},
 {'average_score': 10.0, 'anime_title': 'One Punch Man'},
 {'average_score': 10.0, 'anime_title': 'Zetsuen no Tempest'},
 {'average_score': 10.0, 'anime_title': 'Byousoku 5 Centimeter'},
 {'average_score': 10.0, 'anime_title': 'Seihou Bukyou Outlaw Star'},
 {'average_score': 10.0, 'anime_title': 'Hataraku Maou-sama!'},
 {'average_score': 10.0, 'anime_title': 'Tanaka-kun wa Itsumo Kedaruge'},
 {'average_score': 10.0, 'anime_title': 'Shirokuma Cafe'},
 {'average_score': 10.0, 'anime_title': 'Little Witch Academia'},
 {'average_score': 10.0, 'anime_title': 'Little Witch Academia (TV)'},
 {'average_score': 10.0, 'anime_title': 'Tekkon Kinkreet'},
 {'average_score': 10.0, 'anime_title': 'Haibane Renmei'},
 {'average_score': 9.5, 'anime_title': 'Higashi no Eden'},
 {'average_score': 9.0, 'anime_title': 'Non Non Biyori'},
 {'average_score': 9.0, 'anime_title': 'Naruto'},
 {'averag

In [46]:
pipeline = [
    {
        "$lookup": {
            "from": "profiles_opt",
            "localField": "profile",
            "foreignField": "profile",
            "as": "profile_info"
        }
    },
    {"$unwind": "$profile_info"},
    {
        "$match": {
            "profile_info.gender": "Female",
            "profile_info.birthday": {"$gte": 1995, "$lte": 2005}
        }
    },
    {
        "$group": {
            "_id": "$anime_uid",
            "total_reviews": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "animes_opt",
            "localField": "_id",
            "foreignField": "uid",
            "as": "anime_info"
        }
    },
    {"$unwind": "$anime_info"},
    {
        "$project": {
            "_id": 0,
            "anime_title": "$anime_info.title",
            "total_reviews": 1
        }
    },
    {"$sort": {"total_reviews": -1}},
    {"$limit": 10}
]

result = list(db.reviews_opt.aggregate(pipeline))
result

[{'total_reviews': 52, 'anime_title': 'Yuri!!! on Ice'},
 {'total_reviews': 52, 'anime_title': 'Yuri!!! on Ice'},
 {'total_reviews': 45, 'anime_title': 'Kuroshitsuji'},
 {'total_reviews': 45, 'anime_title': 'Kuroshitsuji'},
 {'total_reviews': 45, 'anime_title': 'Kuroshitsuji'},
 {'total_reviews': 45, 'anime_title': 'Kuroshitsuji'},
 {'total_reviews': 45, 'anime_title': 'Kuroshitsuji'},
 {'total_reviews': 45, 'anime_title': 'Kuroshitsuji'},
 {'total_reviews': 45, 'anime_title': 'Kuroshitsuji'},
 {'total_reviews': 28, 'anime_title': 'Charlotte'}]

In [47]:
pipeline = [
    {
        "$lookup": {
            "from": "profiles_opt",
            "localField": "profile",
            "foreignField": "profile",
            "as": "profile_info"
        }
    },
    {"$unwind": "$profile_info"},
    {
        "$group": {
            "_id": {
                "anime_uid": "$anime_uid",
                "gender": "$profile_info.gender"
            },
            "average_score": {"$avg": "$score"}
        }
    },
    {
        "$group": {
            "_id": "$_id.anime_uid",
            "scores_by_gender": {
                "$push": {
                    "gender": "$_id.gender",
                    "average_score": "$average_score"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "anime_uid": "$_id",
            "score_diff": {
                "$abs": {
                    "$subtract": [
                        {"$arrayElemAt": ["$scores_by_gender.average_score", 1]},
                        {"$arrayElemAt": ["$scores_by_gender.average_score", 0]}
                    ]
                }
            }
        }
    },
    {
        "$lookup": {
            "from": "animes_opt",
            "localField": "anime_uid",
            "foreignField": "uid",
            "as": "anime_info"
        }
    },
    {"$unwind": "$anime_info"},
    {
        "$project": {
            "anime_title": "$anime_info.title",
            "score_diff": 1
        }
    },
    {"$sort": {"score_diff": -1}},
    {"$limit": 4}
]
result = list(db.reviews_opt.aggregate(pipeline))
result

[{'score_diff': 6.538461538461538, 'anime_title': 'Eyeshield 21'},
 {'score_diff': 6.538461538461538, 'anime_title': 'Eyeshield 21'},
 {'score_diff': 6.538461538461538, 'anime_title': 'Eyeshield 21'},
 {'score_diff': 6.538461538461538, 'anime_title': 'Eyeshield 21'}]